In [1]:
"""
Limpieza de archivos meteorológicos para simulaciones solares.

Este script:
1. Carga archivos CSV con datos meteorológicos potencialmente corruptos.
2. Verifica y corrige valores fuera de los rangos físicos definidos.
3. Aplica interpolación y relleno a valores perdidos o atípicos.
4. Elimina filas completamente vacías.
5. Exporta una versión limpia del archivo con sufijo "_clean.csv".

Rangos físicos basados en umbrales típicos de sensores meteorológicos.
Requiere: pandas, numpy, os
"""

import pandas as pd
import numpy as np
import os

# === Rangos físicos aceptables por variable ===
rango_valido = {
    'GHI': (0, 1200),         # Irradiancia global horizontal [W/m²]
    'DNI': (0, 1000),         # Irradiancia directa normal [W/m²]
    'DHI': (0, 600),          # Irradiancia difusa horizontal [W/m²]
    'Tdry': (-40, 60),        # Temperatura ambiente [°C]
    'Tdew': (-40, 60),        # Punto de rocío [°C]
    'RH': (0, 100),           # Humedad relativa [%]
    'Pres': (800, 1100),      # Presión atmosférica [hPa]
    'Wspd': (0, 60),          # Velocidad del viento [m/s]
    'Wdir': (0, 360),         # Dirección del viento [°]
    'Snow Depth': (0, 200)    # Profundidad de nieve [cm]
}

# === Lista de archivos a limpiar ===
archivos = [
    'calama_corrupted.csv',
    'salvador_corrupted.csv',
    'Vallenar_corrupted.csv'
]

# === Función de limpieza de un archivo ===
def limpiar_archivo(nombre_archivo):
    """
    Realiza limpieza básica y reemplazo de valores fuera de rango en un archivo CSV meteorológico.
    - Reemplaza valores extremos por NaN.
    - Interpola hasta 3 valores consecutivos.
    - Aplica forward-fill y backward-fill.
    - Guarda archivo limpio con sufijo "_clean.csv".
    """
    print(f"🔍 Procesando: {nombre_archivo}")
    
    # Cargar archivo
    df = pd.read_csv(nombre_archivo)
    original_cols = df.columns.tolist()

    # Cortar columnas sobrantes si existen (máximo 15)
    df = df[df.columns[:15]]

    # Eliminar filas completamente vacías
    df.dropna(how='all', inplace=True)

    # Procesar cada variable con umbral definido
    for col, (vmin, vmax) in rango_valido.items():
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            # Reemplazar valores fuera de rango por NaN
            df.loc[(df[col] < vmin) | (df[col] > vmax), col] = np.nan
            # Interpolar valores perdidos
            df[col] = df[col].interpolate(limit=3)
            # Rellenar extremos si quedan NaN
            df[col] = df[col].ffill().bfill()

    # Guardar archivo limpio
    nombre_limpio = nombre_archivo.replace("_corrupted", "_clean")
    df.to_csv(nombre_limpio, index=False)
    print(f"✅ Guardado: {nombre_limpio} ({len(df)} filas)\n")

# === Ejecutar limpieza en todos los archivos ===
for archivo in archivos:
    if os.path.exists(archivo):
        limpiar_archivo(archivo)
    else:
        print(f"❌ No se encontró el archivo: {archivo}")


🔍 Procesando: calama_corrupted.csv
✅ Guardado: calama_clean.csv (8760 filas)

🔍 Procesando: salvador_corrupted.csv
✅ Guardado: salvador_clean.csv (8760 filas)

🔍 Procesando: Vallenar_corrupted.csv
✅ Guardado: Vallenar_clean.csv (8760 filas)

